In [ ]:
import pygplates
import sys, os
import glob
import PlateFrameRasters

#************
#change the following variables to generate your plate ref frame grids
#the domain_points need to be reassign plate ids using your rotation model. this can be done with GPlates.
#the time_list and raster_filenames must have the same length
#for each raster in raster_filenames, the program will produce plate ref frame raster at the corresponding time in time_list
time_list = [0,69,140]
raster_filenames = [
    './rasters/M7/MantleFrame/M7.0.Ma.nc',
    './rasters/M7/MantleFrame/M7.69.Ma.nc',
    './rasters/M7/MantleFrame/M7.140.Ma.nc'
]
out_dir = './rasters/M7/PlateFrame/'
multipoint_feature_collection_name = './reconstructions/Global_Model_WD_Internal_Release_2015_v2/domain_points/lat_lon_velocity_domain_720_1440.shp'
rotation_filenames = glob.glob('./reconstructions/Global_Model_WD_Internal_Release_2015_v2/*.rot')
#*************

rotation_model = pygplates.RotationModel(rotation_filenames)
file_registry = pygplates.FeatureCollectionFileFormatRegistry()
multipoint_feature_collection = file_registry.read(multipoint_feature_collection_name)

if not os.path.isdir(out_dir):
    os.system('mkdir --parents ' + out_dir)


selected_features = multipoint_feature_collection.get(
    lambda feature: feature.get_reconstruction_plate_id()!=0,
    pygplates.FeatureReturn.all)

multipoint_feature_collection = pygplates.FeatureCollection(selected_features)

# To Get Plate Frame Grids 
outfile_stem = out_dir
PlateFrameRasters.GeneratePlateReferenceFramesXYZ(
    rotation_model, 
    time_list, 
    raster_filenames, 
    multipoint_feature_collection,
    outfile_stem)
print 'done'


In [ ]:
import numpy as np
from netCDF4 import Dataset
from mpl_toolkits.basemap import Basemap, cm
import matplotlib.pyplot as plt
%matplotlib inline

#this code cell is for test purpose only.
#change the InputGridFile to display the generated grids
InputGridFile = './rasters/M7/PlateFrame/PlateFrameGrid69.00.nc'

def cmap_discretize(cmap, N):
    #http://wiki.scipy.org/Cookbook/Matplotlib/ColormapTransformations
    """Return a discrete colormap from the continuous colormap cmap.     
    cmap: colormap instance, eg. cm.jet. 
    N: number of colors.
    
    Example
    x = resize(arange(100), (5,100))
    djet = cmap_discretize(cm.jet, 5)
    imshow(x, cmap=djet)
    """
    import matplotlib
    
    if type(cmap) == str:
        cmap = get_cmap(cmap)
    colors_i = np.concatenate((np.linspace(0, 1., N), (0.,0.,0.,0.)))
    colors_rgba = cmap(colors_i)
    indices = np.linspace(0, 1., N+1)
    cdict = {}
    for ki,key in enumerate(('red','green','blue')):
        cdict[key] = [ (indices[i], colors_rgba[i-1,ki], colors_rgba[i,ki]) for i in xrange(N+1) ]
    
    # Return colormap object.
    return matplotlib.colors.LinearSegmentedColormap(cmap.name + "_%d"%N, cdict, 1024)


data = Dataset(InputGridFile,'r')
gridX,gridY = np.meshgrid(data.variables['lon'][:],data.variables['lat'][:])
gridZ = data.variables['z'][:]

# create figure and axes instances
fig = plt.figure(figsize=(16,12),dpi=300,frameon=False)
ax = fig.add_axes([0.1,0.1,0.8,0.8])
ax.axis('off')

m  = Basemap(projection='cyl',llcrnrlat=-90,urcrnrlat=90,\
        llcrnrlon=-180,urcrnrlon=180,resolution='c')

# draw coastlines, state and country boundaries, edge of map.
m.drawcoastlines()
m.drawcountries()
parallels = np.arange(-90.,90,30.)
meridians = np.arange(-180.,360.,30.)

x, y = m(gridX, gridY) # compute map proj coordinates.

clip_path = m.drawmapboundary()
s = m.transform_scalar(gridZ, gridX[0,:], gridY[:,0], 1000, 500)
cm = cmap_discretize(plt.cm.RdBu_r, 30)
im = m.imshow(s, cmap=cm, clip_path=clip_path, vmin=-1500, vmax=1500)

# add colorbar.
cbar = m.colorbar(im, location="bottom")
cbar.set_label('Dynamic Topography (m)')
# add title
plt.title('Plate ref frame grid')
plt.show()

